## 모듈 import

In [1]:
#모듈 import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
from pycaret.classification  import *

# Waeing
import sys, warnings
if not sys.warnoptions: warnings.simplefilter("ignore")

# Label Encoding
from sklearn.preprocessing import LabelEncoder

print('Pandas : %s'%(pd.__version__))
print('Numpy : %s'%(np.__version__))
print('matplotlib : %s'%(matplotlib.__version__))
print('pycaret : %s'%(pycaret.__version__))
!python --version

Pandas : 1.3.2
Numpy : 1.19.5
matplotlib : 3.3.4
pycaret : 2.3.3
Python 3.8.8


사용한 버전  
Pandas : 1.3.2  
Numpy : 1.19.5  
matplotlib : 3.3.4  
pycaret : 2.3.3  
Python 3.8.8

## data 불러오기

In [2]:
# 데이터가 저장된 경로 설정
data_path = './daplatformers2021/'

# 데이터 불러오기
train = pd.read_csv(data_path + 'trainset.csv')
test = pd.read_csv(data_path + 'testset_final.csv')
submission = pd.read_csv(data_path + 'sample_submission_final.csv')

In [3]:
train = train.drop(columns=['매니저최초가입일', '매니저최초서비스일', '매니저주소'])

### 전처리

In [4]:
data = pd.concat([train,test]).reset_index(drop=True);data.head(3)

,SEQ,접수일,접수시각,장기서비스여부,최초서비스일,전체회차,현재회차,서비스일자,서비스시작시간,서비스종료시간,기존고객여부,결재형태,서비스주소,주거형태,평수,고객가입일,반려동물,부재중여부,우선청소,쿠폰사용여부,매니저생년월일,매니저성별,매니저사용휴대폰,매니저이동방법,근무가능지역,CS교육이수여부,청소교육이수여부,부재중서비스가능여부,추천인여부,매칭성공여부
0,T06420,2019-07-09,NaN,1,2019-07-15,3,3,2019-07-29,9:00:00,13:00:00,1,무통장입금,충남 천안시,일반주택,NaN,2019-04-22,NaN,NaN,NaN,0,1956-04-19,0,안드로이드,대중교통,서울,0.0,0.0,1,0,0.0
1,T15430,2019-07-12,NaN,1,2019-07-15,4,3,2019-08-20,10:00:00,16:00:00,1,무통장입금,충남 천안시,일반주택,NaN,2019-05-21,없음,NaN,NaN,0,1976-06-10,0,안드로이드,대중교통,천안/아산,1.0,1.0,1,0,0.0
2,T23790,2020-11-18,NaN,1,2020-11-23,10,6,2020-12-11,9:00:00,13:00:00,1,신용카드,충남 천안시,일반주택,40평대이상,2020-10-14,없음,0.0,NaN,0,1970-04-05,0,안드로이드,대중교통,천안/아산,0.0,0.0,0,0,0.0


- 결측치 채움

In [5]:
# 결측치 0으로 채움
data['청소교육이수여부'] = data['청소교육이수여부'].fillna(0)
data['CS교육이수여부'] = data['CS교육이수여부'].fillna(0)
data['부재중여부'] = data['부재중여부'].fillna(0)
# 결측치 최빈값으로 채움
data['매니저이동방법'] = data['매니저이동방법'].fillna(data['매니저이동방법'].mode()[0])
data['매니저사용휴대폰'] = data['매니저사용휴대폰'].fillna(data['매니저사용휴대폰'].mode()[0])
data['고객가입일'] = data['고객가입일'].fillna(data['고객가입일'].mode()[0]) #데이터 바뀌고 test에 생긴 에러
data['결재형태'] = data['결재형태'].fillna(data['결재형태'].mode()[0]) #데이터 바뀌고 test에 생긴 에러

data['접수시각'] = data['접수시각'].fillna(data['접수시각'].mode()[0])
data['반려동물'] = data['반려동물'].fillna('없음')
# 결측치 판단하여 없음으로 채움
data['우선청소'] = data['우선청소'].fillna('없음')

### 평수 평균 확인 -> 31.xx -> 30평대로 채움
data['평수'] = data['평수'].fillna('30평대')

In [6]:
#test와 train의 구조를 맞춰줌
data['매니저생년월일'] = data['매니저생년월일'].astype('str').apply(lambda x: x[0:4])

In [7]:
# 저장해놓기
data_SEQ = data.SEQ
# 인코딩을 위하여 제거 / 이후 나중에 병합함
y_data = data['매칭성공여부']
data = data.drop(columns = ['SEQ','매칭성공여부'])

- Encoding

In [8]:
#범주형변수 수동으로 분리
data[['장기서비스여부','기존고객여부','부재중여부','쿠폰사용여부','매니저성별',
         'CS교육이수여부','청소교육이수여부','부재중서비스가능여부','추천인여부']
     ] = data[['장기서비스여부','기존고객여부','부재중여부','쿠폰사용여부','매니저성별',
         'CS교육이수여부','청소교육이수여부','부재중서비스가능여부','추천인여부'
     ]].astype('str')

# 범주형 변수와 수치형 변수를 분리
cat_features = data.select_dtypes(include=['object']).columns.to_list()
num_features = data.select_dtypes(exclude='object').columns.to_list() 

#원핫인코딩
data_cat = pd.get_dummies(data[cat_features])
data = pd.concat([data,data_cat],axis=1).drop(columns=[f for f in cat_features])

In [9]:
#위에서 떼어놨던 피처 병합
data = pd.concat([data_SEQ, data],axis=1)
data = pd.concat([data, y_data],axis=1);data.head(3)

,SEQ,전체회차,현재회차,접수일_2019-04-09,접수일_2019-04-12,접수일_2019-04-15,접수일_2019-04-16,접수일_2019-04-22,접수일_2019-04-25,접수일_2019-04-29,접수일_2019-04-30,접수일_2019-05-03,접수일_2019-05-08,접수일_2019-05-09,접수일_2019-05-10,접수일_2019-05-13,접수일_2019-05-15,접수일_2019-05-16,접수일_2019-05-20,접수일_2019-05-23,접수일_2019-05-24,접수일_2019-05-27,접수일_2019-05-28,접수일_2019-05-29,접수일_2019-05-30,접수일_2019-05-31,접수일_2019-06-04,접수일_2019-06-05,접수일_2019-06-07,접수일_2019-06-10,접수일_2019-06-11,접수일_2019-06-14,접수일_2019-06-17,접수일_2019-06-18,접수일_2019-06-19,접수일_2019-06-20,접수일_2019-06-21,접수일_2019-06-24,접수일_2019-06-25,접수일_2019-06-26,접수일_2019-06-27,접수일_2019-06-28,접수일_2019-07-01,접수일_2019-07-02,접수일_2019-07-03,접수일_2019-07-04,접수일_2019-07-05,접수일_2019-07-08,접수일_2019-07-09,접수일_2019-07-10,접수일_2019-07-11,접수일_2019-07-12,접수일_2019-07-15,접수일_2019-07-16,접수일_2019-07-17,접수일_2019-07-18,접수일_2019-07-19,접수일_2019-07-20,접수일_2019-07-21,접수일_2019-07-22,접수일_2019-07-23,접수일_2019-07-24,접수일_2019-07-25,접수일_2019-07-26,접수일_2019-07-27,접수일_2019-07-28,접수일_2019-07-29,접수일_2019-07-30,접수일_2019-07-31,접수일_2019-08-01,접수일_2019-08-02,접수일_2019-08-05,접수일_2019-08-06,접수일_2019-08-07,접수일_2019-08-08,접수일_2019-08-09,접수일_2019-08-12,접수일_2019-08-13,접수일_2019-08-14,접수일_2019-08-17,접수일_2019-08-19,접수일_2019-08-20,접수일_2019-08-21,접수일_2019-08-22,접수일_2019-08-23,접수일_2019-08-24,접수일_2019-08-26,접수일_2019-08-27,접수일_2019-08-28,접수일_2019-08-29,접수일_2019-08-30,접수일_2019-09-02,접수일_2019-09-03,접수일_2019-09-04,접수일_2019-09-05,접수일_2019-09-06,접수일_2019-09-08,접수일_2019-09-09,접수일_2019-09-10,접수일_2019-09-11,접수일_2019-09-16,접수일_2019-09-17,접수일_2019-09-18,접수일_2019-09-19,접수일_2019-09-20,접수일_2019-09-23,접수일_2019-09-24,접수일_2019-09-25,접수일_2019-09-26,접수일_2019-09-27,접수일_2019-09-30,접수일_2019-10-01,접수일_2019-10-02,접수일_2019-10-03,접수일_2019-10-04,접수일_2019-10-06,접수일_2019-10-07,접수일_2019-10-08,접수일_2019-10-09,접수일_2019-10-10,접수일_2019-10-11,접수일_2019-10-12,접수일_2019-10-15,접수일_2019-10-16,접수일_2019-10-17,접수일_2019-10-18,접수일_2019-10-19,접수일_2019-10-21,접수일_2019-10-22,접수일_2019-10-23,접수일_2019-10-24,접수일_2019-10-26,접수일_2019-10-28,접수일_2019-10-29,접수일_2019-10-31,접수일_2019-11-01,접수일_2019-11-13,접수일_2019-11-21,접수일_2019-11-22,접수일_2019-11-25,접수일_2019-11-28,접수일_2019-11-29,접수일_2019-12-02,접수일_2019-12-04,접수일_2019-12-09,접수일_2019-12-11,접수일_2019-12-17,접수일_2019-12-18,접수일_2019-12-21,접수일_2019-12-23,접수일_2019-12-24,접수일_2019-12-26,접수일_2019-12-27,접수일_2019-12-28,접수일_2019-12-31,접수일_2020-01-01,접수일_2020-01-02,접수일_2020-01-04,접수일_2020-01-10,접수일_2020-01-13,접수일_2020-01-14,접수일_2020-01-15,접수일_2020-01-16,접수일_2020-01-17,접수일_2020-01-18,접수일_2020-01-20,접수일_2020-01-21,접수일_2020-01-23,접수일_2020-01-28,접수일_2020-01-29,접수일_2020-01-30,접수일_2020-01-31,접수일_2020-02-03,접수일_2020-02-04,접수일_2020-02-05,접수일_2020-02-06,접수일_2020-02-07,접수일_2020-02-10,접수일_2020-02-11,접수일_2020-02-13,접수일_2020-02-14,접수일_2020-02-17,접수일_2020-02-18,접수일_2020-02-19,접수일_2020-02-24,접수일_2020-02-25,접수일_2020-02-27,접수일_2020-03-03,접수일_2020-03-06,접수일_2020-03-09,접수일_2020-03-12,접수일_2020-03-16,접수일_2020-03-18,접수일_2020-03-26,접수일_2020-03-27,접수일_2020-03-29,접수일_2020-03-30,접수일_2020-03-31,접수일_2020-04-02,접수일_2020-04-07,접수일_2020-04-08,접수일_2020-04-09,접수일_2020-04-13,접수일_2020-04-14,접수일_2020-04-22,접수일_2020-04-23,접수일_2020-04-26,접수일_2020-04-27,접수일_2020-04-29,접수일_2020-05-01,접수일_2020-05-07,접수일_2020-05-11,접수일_2020-05-17,접수일_2020-05-18,접수일_2020-05-19,접수일_2020-05-20,접수일_2020-05-21,접수일_2020-05-24,접수일_2020-05-25,접수일_2020-05-26,접수일_2020-05-27,접수일_2020-05-28,접수일_2020-05-29,접수일_2020-05-30,접수일_2020-06-01,접수일_2020-06-02,접수일_2020-06-03,접수일_2020-06-04,접수일_2020-06-06,접수일_2020-06-07,접수일_2020-06-08,접수일_2020-06-10,접수일_2020-06-14,접수일_2020-06-15,접수일_2020-06-16,접수일_2020-06-17,접수일_2020-06-18,접수일_2020-06-19,접수일_2020-06-21,접수일_2020-06-22,접수일_2020-06-23,접수일_2020-06-24,접수일_2020-06-25,접수일_2020-06-26,접수일_2020-06-28,접수일_2020-06-30,접수일_2020-07-01,접수일_2020-07-02,접수일_2020-07-05,접수일_2020-07-06,...,고객가입일_2020-07-08,고객가입일_2020-07-10,고객가입일_2020-07-15,고객가입일_2020-07-20,고객가입일_2020-07-21,고객가입일_2020-07-27,고객가입일_2020-07-28,고객가입일_2020-07-29,고객가입일_2020-07-30,고객가입일_2020-08-04,고객가입일_2020-08-05,고객가입일_2020-08-07,고객가입일_2020-08-10,고객가입일_2020-08-14,고객가입일_2020-08-15,고객가입일_2020-08-17,고객가입

- Data split

In [10]:
#train과 test를 분리
train = data.iloc[:23009,:]
test = data.iloc[23009:,:]
test = test.drop(columns = ['매칭성공여부']).reset_index(drop=True)

## Modeling with pycaret

- Model setting

In [11]:
train = train.drop(columns=['SEQ'])

In [12]:
cl = setup(train, 
            preprocess = False, 
            train_size = 0.999,  
            target = '매칭성공여부', 
            silent = True, 
            use_gpu = True, 
            numeric_features=['전체회차','현재회차'], 
            session_id = 42,
            fold_shuffle = True
            )

,Description,Value
0,session_id,42
1,Target,매칭성공여부
2,Target Type,Binary
3,Label Encoded,"0.0: 0, 1.0: 1"
4,Original Data,"(23009, 2825)"
5,Missing Values,False
6,Numeric Features,2824
7,Categorical Features,0
8,Transformed Train Set,"(22985, 2824)"
9,Transformed Test Set,"(24, 2824)"


In [13]:
top = compare_models(sort='AUC', fold=5, n_select = 1, include=['catboost'])

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
catboost,CatBoost Classifier,0.9389,0.9557,0.3790,0.9536,0.5420,0.5157,0.5796,12.0780


- Create model

In [14]:
#catboost
catboost = create_model('catboost')
#hyperparameter tuning
catboost = tune_model(catboost, optimize = 'AUC', choose_better = True, fold = 5, n_iter = 100)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.9626,0.9782,0.6355,0.9588,0.7644,0.7450,0.7636
1,0.9595,0.9759,0.6014,0.9600,0.7395,0.7188,0.7420
2,0.9604,0.9789,0.6068,0.9674,0.7458,0.7256,0.7488
3,0.9630,0.9791,0.6432,0.9561,0.7690,0.7498,0.7672
4,0.9545,0.9684,0.5477,0.9602,0.6975,0.6749,0.7061
Mean,0.9600,0.9761,0.6069,0.9605,0.7433,0.7228,0.7455
SD,0.0030,0.0040,0.0337,0.0038,0.0254,0.0266,0.0218


- Fit & predict

In [15]:
model = catboost
pred_holdout = predict_model(model, test)

In [16]:
#pycaret에서는 score이 label을 맞출 확률이기때문에 output을 제출 양식에 맞게 바꿔줍니다
pred = pd.Series([pred_holdout['Score'][i] if pred_holdout['Label'][i] =='1.0' else (1-pred_holdout['Score'][i]) for i in range(len(pred_holdout['Score']))]
)

In [17]:
submission['pred'] = pred

### Make submission 

In [18]:
fname = 'cat_0.83196.csv'
submission.to_csv(fname, index=False)
print("'{}' is ready to submit." .format(fname))

'cat_0.83196.csv' is ready to submit.


# END